In [7]:
using ModelingToolkit, OrdinaryDiffEq, DifferentialEquations, LinearAlgebra, SparseArrays

┌ Info: Precompiling ModelingToolkit [961ee093-0014-501f-94e3-6117800e7a78]
└ @ Base loading.jl:1260
ERROR: LoadError: ArgumentError: Package ModelingToolkit does not have LabelledArrays in its dependencies:
- If you have ModelingToolkit checked out for development and have
  added LabelledArrays as a dependency but haven't updated your primary
  environment's manifest file, try `Pkg.resolve()`.
- Otherwise you may need to report an issue with ModelingToolkit
Stacktrace:
 [1] require(::Module, ::Symbol) at ./loading.jl:905
 [2] include(::Module, ::String) at ./Base.jl:377
 [3] top-level scope at none:2
 [4] eval at ./boot.jl:331 [inlined]
 [5] eval(::Expr) at ./client.jl:449
 [6] top-level scope at ./none:3
in expression starting at /mnt/storage_1/users/cullinanen/.julia/packages/ModelingToolkit/TAGTl/src/ModelingToolkit.jl:4


ErrorException: Failed to precompile ModelingToolkit [961ee093-0014-501f-94e3-6117800e7a78] to /mnt/storage_1/users/cullinanen/.julia/compiled/v1.4/ModelingToolkit/Qmdqu_19dC0.ji.

In [5]:
using PyPlot, NNlib,  ImageFiltering, Images, MEngProject, MEngProject.LamKernels, MEngProject.Laminart, MEngProject.Utils, BenchmarkTools

┌ Info: Precompiling Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1260
ERROR: LoadError: ArgumentError: Package ImageTransformations does not have Rotations in its dependencies:
- If you have ImageTransformations checked out for development and have
  added Rotations as a dependency but haven't updated your primary
  environment's manifest file, try `Pkg.resolve()`.
- Otherwise you may need to report an issue with ImageTransformations
Stacktrace:
 [1] require(::Module, ::Symbol) at ./loading.jl:905
 [2] include(::Module, ::String) at ./Base.jl:377
 [3] top-level scope at none:2
 [4] eval at ./boot.jl:331 [inlined]
 [5] eval(::Expr) at ./client.jl:449
 [6] top-level scope at ./none:3
in expression starting at /mnt/storage_1/users/cullinanen/.julia/packages/ImageTransformations/J3Mno/src/ImageTransformations.jl:5
ERROR: LoadError: Failed to precompile ImageTransformations [02fcd773-0e25-5acc-982a-7f6622650795] to /mnt/storage_1/users/cullinanen/.julia/compiled/v1.4/I

ErrorException: Failed to precompile Images [916415d5-f1e6-5110-898d-aaa5f9f070e0] to /mnt/storage_1/users/cullinanen/.julia/compiled/v1.4/Images/H8Vxc_19dC0.ji.

In [3]:
using CUDA, Test

┌ Info: Precompiling CUDA [052768ef-5323-5732-b1bb-66c8b64840ba]
└ @ Base loading.jl:1260
